<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\stive\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\stive\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# Cargar datos
path = 'reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
def pre_procesado(text):
    text = text.lower()
    text = re.sub(r"[\W\d]+", " ", text)
    text = [p for p in text.split() if p not in stopwords]
    return text 


In [4]:

data['pre_procesado'] = data['review_body'].apply(lambda text: pre_procesado(text))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pre_procesado
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [6]:
dictionary = Dictionary(data['pre_procesado'].values)
dictionary.filter_extremes(no_below=5, no_above=0.5) 
corpus = [dictionary.doc2bow(text) for text in data['pre_procesado'].values]
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 11, passes=150)
model.print_topics(num_words=10)

[(0,
  '0.050*"perfecto" + 0.048*"producto" + 0.045*"llegado" + 0.043*"llegó" + 0.038*"llego" + 0.030*"tiempo" + 0.024*"día" + 0.020*"sido" + 0.019*"nunca" + 0.019*"roto"'),
 (1,
  '0.071*"juego" + 0.025*"jugar" + 0.024*"hijo" + 0.021*"versión" + 0.021*"si" + 0.018*"día" + 0.018*"todavía" + 0.016*"puede" + 0.016*"q" + 0.013*"llegado"'),
 (2,
  '0.043*"si" + 0.035*"producto" + 0.026*"mas" + 0.024*"comprar" + 0.023*"precio" + 0.020*"después" + 0.017*"original" + 0.017*"siempre" + 0.015*"correcto" + 0.015*"solo"'),
 (3,
  '0.051*"bien" + 0.041*"cadena" + 0.034*"calidad" + 0.030*"tamaño" + 0.029*"puede" + 0.027*"precio" + 0.027*"pequeños" + 0.023*"tan" + 0.023*"grandes" + 0.022*"bonitos"'),
 (4,
  '0.039*"bonito" + 0.037*"foto" + 0.027*"plata" + 0.026*"pendientes" + 0.025*"tal" + 0.025*"bonitos" + 0.022*"precio" + 0.022*"bien" + 0.022*"gustado" + 0.019*"parece"'),
 (5,
  '0.099*"bien" + 0.048*"si" + 0.022*"ps" + 0.019*"caja" + 0.017*"jugar" + 0.016*"precio" + 0.016*"problema" + 0.016*"solo

###  `[25 pts]` Punto 3: Visualización de LDA

In [ ]:
lda_display = pyLDAvis.gensim_models.prepare(model,corpus,dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

###  Concluciones
- Para el primer taller fue buenos que tuviera las etiquetas, ya que con esas se podia clasificar dependiendo su parecido con la uno o la otra
- Para el segundo taller separe en algunos conjustos, pero dependiendo el numerjo de grupos, cada uno iba tienendo caracteristicas mas especificas
- En este ultimo taller depende la clisificacion de los parametros que le pasemos, eso hace que cambie la segmentacion de los datos


